# Taxi data
First of all, we explore the dataset of the [taxi trips in NYC](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page) with pandas.

In [24]:
import pandas as pd

trip_data = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet"
)

trip_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5,NaN
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,NaN
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,NaN
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,NaN
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,NaN


The data follows the data dictionary as given [here](https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf). For the question that we would like to answer (does rain have an influence on the amount of taxi trips?), we need a couple of columns, namely:
* tpep_pickup_datetime -> pickup_timestamp
* tpep_dropoff_datetime -> dropoff_timestamp
* passenger_count -> passenger_count
* trip_distance -> trip_distance
* PULocationID -> pickup_location_id
* DOLocationID -> drop_off_location_id
* RatecodeID -> rate_code
* payment_type -> payment_type
* fare_amount -> fare_amount
* total_amount -> paid_amount

In [5]:
columns_to_keep = "tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, PULocationID, DOLocationID, RatecodeID, payment_type, fare_amount, total_amount".split(
    ", "
)

trip_data[columns_to_keep].set_axis(
    "pickup_timestamp, dropoff_timestamp, passenger_count, trip_distance, pickup_location_id, drop_off_location_id, rate_code, payment_type, fare_amount, paid_amount".split(
        ", "
    ),
    axis=1,
)

trip_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5,NaN
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,NaN
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,NaN
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,NaN
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,NaN


We do a simple count to check if we have null values and in which columns those appear.

In [7]:
trip_data.count()

VendorID                 1369769
tpep_pickup_datetime     1369769
tpep_dropoff_datetime    1369769
passenger_count          1271417
trip_distance            1369769
RatecodeID               1271417
store_and_fwd_flag       1271417
PULocationID             1369769
DOLocationID             1369769
payment_type             1369769
fare_amount              1369769
extra                    1369769
mta_tax                  1369769
tip_amount               1369769
tolls_amount             1369769
improvement_surcharge    1369769
total_amount             1369769
congestion_surcharge     1271417
airport_fee                    5
dtype: int64

Thus, we find that there are some rides where we do not have the passenger_count. When we read the data dictionary, we find that the passenger_count is filled in by the driver. Thus, we dive a bit deeper if we are able to discard these from our dataset. When the passenger_count is zero and the trip_distance is zero, then we must have a faulty ride. Let us check if this is true.

In [8]:
trip_data[trip_data["passenger_count"].isna()]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1271417,2,2021-01-01 00:06:47,2021-01-01 00:54:51,NaN,15.95,NaN,None,49,86,0,45.65,0.00,0.5,2.75,0.00,0.3,49.20,NaN,NaN
1271418,2,2021-01-01 00:43:03,2021-01-01 01:20:56,NaN,18.28,NaN,None,45,15,0,59.87,0.00,0.5,2.75,0.00,0.3,63.42,NaN,NaN
1271419,1,2021-01-01 00:35:18,2021-01-01 00:40:52,NaN,0.00,NaN,None,262,237,0,4.20,0.50,0.5,2.80,0.00,0.3,12.80,NaN,NaN
1271420,2,2021-01-01 00:38:00,2021-01-01 00:51:00,NaN,3.04,NaN,None,137,142,0,18.34,0.00,0.5,2.75,0.00,0.3,21.89,NaN,NaN
1271421,2,2021-01-01 00:13:47,2021-01-01 00:39:18,NaN,14.65,NaN,None,226,219,0,44.62,0.00,0.5,2.75,0.00,0.3,48.17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1369764,2,2021-01-31 23:03:00,2021-01-31 23:33:00,NaN,8.89,NaN,None,229,181,0,27.78,0.00,0.5,7.46,0.00,0.3,38.54,NaN,NaN
1369765,2,2021-01-31 23:29:00,2021-01-31 23:51:00,NaN,7.43,NaN,None,41,70,0,32.58,0.00,0.5,0.00,6.12,0.3,39.50,NaN,NaN
1369766,2,2021-01-31 23:25:00,2021-01-31 23:38:00,NaN,6.26,NaN,None,74,137,0,16.85,0.00,0.5,3.90,0.00,0.3,24.05,NaN,NaN
1369767,6,2021-01-31 23:01:06,2021-02-01 00:02:03,NaN,19.70,NaN,None,265,188,0,53.68,0.00,0.5,0.00,0.00,0.3,54.48,NaN,NaN


The trip_distance is not zero when the passenger_count is zero. We assume that the passenger_count is zero when the driver did not fill in the passenger_count. Therefore, we leave this data in the set for analysis. If we were to delete this data, we might bias our dataset.

To conclude, we describe the dataframe to see if we find anything peculiar.

In [9]:
trip_data.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,1.369769e+06,1.271417e+06,1.369769e+06,1.271417e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.369769e+06,1.271417e+06,5.0
mean,1.721725e+00,1.411508e+00,4.631983e+00,1.035081e+00,1.652474e+02,1.614957e+02,1.188578e+00,1.209663e+01,9.705133e-01,4.930412e-01,1.918098e+00,2.477473e-01,2.969412e-01,1.747440e+01,2.239047e+00,0.0
std,5.925347e-01,1.059831e+00,3.939037e+02,5.994830e-01,6.783854e+01,7.210795e+01,5.776546e-01,1.291337e+01,1.231258e+00,7.632059e-02,2.597151e+00,1.672761e+00,4.222168e-02,1.469342e+01,7.989435e-01,0.0
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-4.900000e+02,-5.500000e+00,-5.000000e-01,-1.000000e+02,-3.112000e+01,-3.000000e-01,-4.928000e+02,-2.500000e+00,0.0
25%,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.240000e+02,1.070000e+02,1.000000e+00,6.000000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.080000e+01,2.500000e+00,0.0
50%,2.000000e+00,1.000000e+00,1.700000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,8.500000e+00,0.000000e+00,5.000000e-01,1.860000e+00,0.000000e+00,3.000000e-01,1.380000e+01,2.500000e+00,0.0
75%,2.000000e+00,1.000000e+00,3.020000e+00,1.000000e+00,2.360000e+02,2.360000e+02,1.000000e+00,1.350000e+01,2.500000e+00,5.000000e-01,2.750000e+00,0.000000e+00,3.000000e-01,1.912000e+01,2.500000e+00,0.0
max,6.000000e+00,8.000000e+00,2.631633e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,6.960500e+03,8.250000e+00,5.000000e-01,1.140440e+03,8.117500e+02,3.000000e-01,7.661280e+03,3.000000e+00,0.0


We seem to find some negative values for the fare_amount and total_amount. Let's see what that is all about.

In [17]:
trip_data[(trip_data["total_amount"] < 0) | (trip_data["fare_amount"] < 0)].describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,6769.000000,6655.000000,6769.000000,6655.000000,6769.000000,6769.000000,6769.000000,6769.000000,6769.000000,6769.000000,6769.000000,6769.000000,6769.000000,6769.000000,6655.000000,0.0
mean,2.001773,1.359279,1.843333,1.122164,161.168858,160.309647,3.309942,-10.600363,-0.258388,-0.475846,0.023154,0.225635,-0.289851,-13.405960,-2.064237,NaN
std,0.084197,0.916893,3.331625,0.580424,68.075608,72.231261,0.846504,15.406329,0.565985,0.140604,1.756834,3.154643,0.077292,16.324601,0.948500,NaN
min,2.000000,1.000000,0.000000,1.000000,3.000000,1.000000,0.000000,-490.000000,-5.500000,-0.500000,-100.000000,-31.120000,-0.300000,-492.800000,-2.500000,NaN
25%,2.000000,1.000000,0.200000,1.000000,107.000000,100.000000,3.000000,-10.000000,-0.500000,-0.500000,0.000000,0.000000,-0.300000,-13.800000,-2.500000,NaN
50%,2.000000,1.000000,0.740000,1.000000,161.000000,162.000000,4.000000,-5.500000,0.000000,-0.500000,0.000000,0.000000,-0.300000,-8.800000,-2.500000,NaN
75%,2.000000,1.000000,1.810000,1.000000,234.000000,234.000000,4.000000,-3.500000,0.000000,-0.500000,0.000000,0.000000,-0.300000,-6.800000,-2.500000,NaN
max,6.000000,6.000000,34.350000,5.000000,265.000000,265.000000,4.000000,-0.050000,5.500000,0.500000,20.000000,27.500000,0.300000,29.000000,0.000000,NaN


It might be good to drop these from the dataset. These might be real trips, but there does seem to be something fishy about them as the rides are fairly short compared to the mean and std of the full dataset. If these are only a few compared to the entire dataset, then we drop them.

In [19]:
len(trip_data[(trip_data["total_amount"] < 0) | (trip_data["fare_amount"] < 0)])/len(trip_data)

0.0049417091494989305

It's lower than 1%. Therefore, we will drop these from our dataset when we build our tables.

Finally, to make sense of the location IDs, we simply explore a small CSV file that comes with the taxi data.

In [25]:
location_ids = pd.read_csv(
    "https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv"
)

location_ids.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


# Weather dataset
To obtain the [weather dataset](https://www.wunderground.com/history/daily/us/ny/new-york-city/KLGA/date/2021-01-01), we'll either have to scrape it or find some other way to obtain the data. Let's reload the page and check the developer tools to see if any API requests are made.

When we look along the API requests, we find one to [api.weather.com](.com/v1/location/KLGA:9:US/observations/historical.json?apiKey=e1f10a1e78da46f5b10a1e78da96f525&units=m&startDate=20210101&endDate=20210101). The values from this API call seem to correspond to the table on the web page. When we look at the arguments of the API call, we find `units` as one of them, which we'll set to `m` instead of `e` to obtain metric values. Thus, we can make API calls to obtain the weather data that we need. We'll do so as follows.

In [21]:
import json
import requests

r = requests.get(
    "https://api.weather.com/v1/location/KLGA:9:US/observations/historical.json?apiKey=e1f10a1e78da46f5b10a1e78da96f525&units=m&startDate=20210101&endDate=20210131"
)
json = r.json()

weather_data = pd.DataFrame.from_dict(json["observations"])

We'll simply drop all the columns where we have NULL values.

In [22]:
weather_data = weather_data.dropna(axis=1)
weather_data.head()

,key,class,expire_time_gmt,obs_id,obs_name,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,...,rh,pressure,vis,wc,wdir_cardinal,wspd,precip_hrly,uv_desc,feels_like,uv_index
0,KLGA,observation,1609487460,KLGA,New York/Laguardia,1609480260,N,3,33,3300,...,57,1029.41,16,0,NNW,13,0.0,Low,0,0
1,KLGA,observation,1609491060,KLGA,New York/Laguardia,1609483860,N,3,33,3300,...,57,1030.42,16,0,N,11,0.0,Low,0,0
2,KLGA,observation,1609494660,KLGA,New York/Laguardia,1609487460,N,3,33,3300,...,59,1030.76,16,0,N,9,0.0,Low,0,0
3,KLGA,observation,1609498260,KLGA,New York/Laguardia,1609491060,N,2,33,3300,...,62,1031.10,16,0,N,7,0.0,Low,0,0
4,KLGA,observation,1609501860,KLGA,New York/Laguardia,1609494660,N,2,33,3300,...,62,1031.44,16,0,NNE,7,0.0,Low,0,0


Finally, we pick out those columns that might be of interest to us.

In [23]:
weather_data = weather_data[
    "valid_time_gmt, temp, dewPt, rh, pressure, wspd, precip_hrly, feels_like, wx_phrase".split(
        ", "
    )
].set_axis(
    "timestamp, temperature, dewpoint_temperature, relative_humidity, pressure, windspeed, precipitation, feels_like, classification".split(
        ", "
    ),
    axis=1,
)

weather_data.head()

,timestamp,temperature,dewpoint_temperature,relative_humidity,pressure,windspeed,precipitation,feels_like,classification
0,1609480260,3,-4,57,1029.41,13,0.0,0,Fair
1,1609483860,3,-4,57,1030.42,11,0.0,0,Fair
2,1609487460,3,-4,59,1030.76,9,0.0,0,Fair
3,1609491060,2,-4,62,1031.10,7,0.0,0,Fair
4,1609494660,2,-4,62,1031.44,7,0.0,0,Fair


The data seems to be fine as is! Let's move on to building the ETL script.